# Fiddler Ranking Model Quick Start Guide

Fiddler offer the ability for your teams to observe you ranking models to understand thier performance and catch issues like data drift before they affect your applications.

# Quickstart: Expedia Search Ranking
The following dataset is coming from Expedia. It includes shopping and purchase data as well as information on price competitiveness. The data are organized around a set of “search result impressions”, or the ordered list of hotels that the user sees after they search for a hotel on the Expedia website. In addition to impressions from the existing algorithm, the data contain impressions where the hotels were randomly sorted, to avoid the position bias of the existing algorithm. The user response is provided as a click on a hotel. From: https://www.kaggle.com/c/expedia-personalized-sort/overview

# 0. Imports

In [4]:
import pandas as pd
import lightgbm as lgb
import numpy as np
import time as time
import datetime

# 1. Connect to Fiddler and Create a Project
First we install and import the Fiddler Python client.

In [5]:
!pip install -q fiddler-client
import fiddler as fdl
print(f"Running client version {fdl.__version__}")

Running client version 1.7.3


Before you can add information about your model with Fiddler, you'll need to connect using our API client.

---

**We need a few pieces of information to get started.**
1. The URL you're using to connect to Fiddler
2. Your organization ID
3. Your authorization token

The latter two of these can be found by pointing your browser to your Fiddler URL and navigating to the **Settings** page.

In [6]:
URL = '' # Make sure to include the full URL (including https://). For example, https://abc.xyz.ai
ORG_ID = ''
AUTH_TOKEN = ''

Next we run the following code block to connect to the Fiddler API.

In [7]:
client = fdl.FiddlerApi(url=URL, org_id=ORG_ID, auth_token=AUTH_TOKEN)

Once you connect, you can create a new project by specifying a unique project ID in the client's `create_project` function.

In [8]:
PROJECT_ID = 'search_ranking'

if not PROJECT_ID in client.list_projects():
    print(f'Creating project: {PROJECT_ID}')
    client.create_project(PROJECT_ID)
else:
    print(f'Project: {PROJECT_ID} already exists')

Project: search_ranking already exists


# 2. Upload the Baseline Dataset

Now we retrieve the Expedia Dataset as a baseline for this model.

In [9]:
df = pd.read_csv("https://media.githubusercontent.com/media/fiddler-labs/fiddler-examples/main/quickstart/data/expedia_baseline_data.csv")
df.head()

,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,...,comp8_rate_percent_diff,click_bool,weekday,week_of_year,hour_time,minute_time,time_epoch,early_night,nans_count,score
0,1,12,187,NaN,NaN,219,893,3,3.5,1,...,NaN,0,3,14,8,32,1.365064e+09,False,20,-0.762692
1,1,12,187,NaN,NaN,219,10404,4,4.0,1,...,NaN,0,3,14,8,32,1.365064e+09,False,22,-1.412878
2,1,12,187,NaN,NaN,219,21315,3,4.5,1,...,NaN,0,3,14,8,32,1.365064e+09,False,20,-1.344691
3,1,12,187,NaN,NaN,219,27348,2,4.0,1,...,5.0,0,3,14,8,32,1.365064e+09,False,17,-2.391245
4,1,12,187,NaN,NaN,219,29604,4,3.5,1,...,NaN,0,3,14,8,32,1.365064e+09,False,20,-0.723813


Fiddler uses this baseline dataset to keep track of important information about your data.
  
This includes **data types**, **data ranges**, and **unique values** for categorical variables.

---

You can construct a `DatasetInfo` object to be used as **a schema for keeping track of this information** by running the following code block.

In [10]:
dataset_info = fdl.DatasetInfo.from_dataframe(df=df, max_inferred_cardinality=100)
dataset_info

,column,dtype,count(possible_values),is_nullable,value_range
0,srch_id,INTEGER,,False,"1 - 1,670"
1,site_id,INTEGER,,False,1 - 34
2,visitor_location_country_id,INTEGER,,False,2 - 224
3,visitor_hist_starrating,FLOAT,,True,2.3 - 5.0
4,visitor_hist_adr_usd,FLOAT,,True,51.77 - 518.3
5,prop_country_id,INTEGER,,False,4 - 230
6,prop_id,INTEGER,,False,"1 - 140,800"
7,prop_starrating,INTEGER,,False,0 - 5
8,prop_review_score,FLOAT,,True,0.0 - 5.0
9,prop_brand_bool,INTEGER,,False,0 - 1


Then use the client's [upload_dataset](https://docs.fiddler.ai/reference/clientupload_dataset) function to send this information to Fiddler!
  
*Just include:*
1. A unique dataset ID
2. The baseline dataset as a pandas DataFrame
3. The [DatasetInfo](https://docs.fiddler.ai/reference/fdldatasetinfo) object you just created

In [11]:
DATASET_ID = 'expedia_data'
client.upload_dataset(project_id=PROJECT_ID,
                      dataset={'baseline': df},
                      dataset_id=DATASET_ID,
                      info=dataset_info)

{'uuid': 'aa1b082f-7d0d-40ed-8b90-57d7352b9486',
 'name': 'Ingestion dataset Upload',
 'info': {'project_name': 'search_ranking',
  'resource_name': 'expedia_data',
  'resource_type': 'DATASET'},
 'status': 'SUCCESS',
 'progress': 100.0,
 'error_message': None,
 'error_reason': None}

# 3. Share Model Metadata and Upload the Model

In [24]:
#create model directory to sotre model files
import os
model_dir = "model"
os.makedirs(model_dir)

### 3.a Adding model metadata to Fiddler
To add a Ranking model you must specify the ModelTask as `RANKING` in the model info object.  

Additionally, you must provide the `group_by` argument that corresponds to the query search id. This `group_by` column should be present either in:
- `features` : if it is used to build and run the model
- `metadata_cols` : if not used by the model 

Optionally, you can give a `ranking_top_k` number (default is 50). This will be the number of results within each query to take into account while computing the performance metrics in monitoring.  

Unless the prediction column was part of your baseline dataset, you must provide the minimum and maximum values predictions can take in a dictionary format (see below).  

If your target is categorical (string), you need to provide the `categorical_target_class_details` argument. If your target is numerical and you don't specify this argument, Fiddler will infer it.   

This will be the list of possible values for the target **ordered**. The first element should be the least relevant target level, the last element should be the most relevant target level.

In [19]:
target = 'click_bool'
features = list(df.drop(columns=['click_bool', 'score']).columns)

model_info = fdl.ModelInfo.from_dataset_info(
    dataset_info=client.get_dataset_info(project_id=PROJECT_ID, dataset_id=DATASET_ID),
    target=target,
    features=features,
    input_type=fdl.ModelInputType.TABULAR,
    model_task=fdl.ModelTask.RANKING,
    outputs={'score':[-5.0, 3.0]},
    group_by='srch_id',
    ranking_top_k=20,
    categorical_target_class_details=[0, 1]
)

# inspect model info and modify as needed
model_info

,column,dtype,count(possible_values),is_nullable,value_range
0,click_bool,INTEGER,,False,0 - 1
,column,dtype,count(possible_values),is_nullable,value_range
0,srch_id,INTEGER,,False,"1 - 1,670"
1,site_id,INTEGER,,False,1 - 34
2,visitor_location_country_id,INTEGER,,False,2 - 224
3,visitor_hist_starrating,FLOAT,,True,2 - 5
4,visitor_hist_adr_usd,FLOAT,,True,51 - 518
5,prop_country_id,INTEGER,,False,4 - 230
6,prop_id,INTEGER,,False,"1 - 140,800"
7,prop_starrating,INTEGER,,False,0 - 5


In [20]:
MODEL_ID = 'expedia_model'

if not MODEL_ID in client.list_models(project_id=PROJECT_ID):
    client.add_model(
        project_id=PROJECT_ID,
        dataset_id=DATASET_ID,
        model_id=MODEL_ID,
        model_info=model_info
    )
else:
    print(f'Model: {MODEL_ID} already exists in Project: {PROJECT_ID}. Please use a different name.')

Model: expedia_model already exists in Project: search_ranking. Please use a different name.


### 3.b Create a Model Wrapper Script

Package.py is the interface between Fiddler’s backend and your model. This code helps Fiddler to understand the model, its inputs and outputs.

You need to implement three parts:
- init: Load the model, and any associated files such as feature transformers.
- transform: If you use some pre-processing steps not part of the model file, transform the data into a format that the model recognizes.
- predict: Make predictions using the model.

In [21]:
%%writefile model/package.py

import pickle
from pathlib import Path
import pandas as pd

PACKAGE_PATH = Path(__file__).parent

class ModelPackage:

    def __init__(self):
        """
         Load the model file and any pre-processing files if needed.
        """
        self.output_columns = ['score']
        
        with open(PACKAGE_PATH / 'model.pkl', 'rb') as infile:
            self.model = pickle.load(infile)
    
    def transform(self, input_df):
        """
        Accepts a pandas DataFrame object containing rows of raw feature vectors. 
        Use pre-processing file to transform the data if needed. 
        In this example we don't need to transform the data.
        Outputs a pandas DataFrame object containing transformed data.
        """
        return input_df
    
    def predict(self, input_df):
        """
        Accepts a pandas DataFrame object containing rows of raw feature vectors. 
        Outputs a pandas DataFrame object containing the model predictions whose column labels 
        must match the output column names in model info.
        """
        transformed_df = self.transform(input_df)
        pred = self.model.predict(transformed_df)
        return pd.DataFrame(pred, columns=self.output_columns)
    
def get_model():
    return ModelPackage()

Overwriting model/package.py


### 3.c Retriving the model files 

To explain a model's inner workigs we need to upload the model artifacts. We will retrive a pre-trained model from the Fiddler Repo that was trained with **lightgbm 2.3.0**

In [22]:
import urllib.request
urllib.request.urlretrieve("https://github.com/fiddler-labs/fiddler-examples/blob/main/quickstart/models/ranking_model.pkl", "model/model.pkl")

('model/model.pkl', <http.client.HTTPMessage at 0x167869f70>)

### 3.d Upload the model files to Fiddler


Now as a final step in the setup you can upload the model artifact files using `add_model_artifact`. 
   - The `model_dir` is the path for the folder containing the model file(s) and the `package.py` from ther last step.

In [ ]:
#Uploading Model files
client.add_model_artifact(model_dir=model_dir, project_id=PROJECT_ID, model_id=MODEL_ID)

# 5. Send Traffic For Monitoring

### 5.a Gather and prepare Production Events
This is the production log file we are going to upload in Fiddler.

In [ ]:
df_logs = pd.read_csv('https://media.githubusercontent.com/media/fiddler-labs/fiddler-examples/main/quickstart/data/expedia_logs.csv')
df_logs.tail()

In [ ]:
df_logs['event_id'] = df_logs['event_id'].apply(str)
#timeshift to move the data to last 29 days
df_logs['time_epoch'] = df_logs['time_epoch'] + (float(time.time()) - df_logs['time_epoch'].max())

For ranking, we need to ingest all events from a given Query ID together. To do that, we need to transform the data to a grouped format.  
You can use the `convert_flat_csv_data_to_grouped` utility function to do the transformation.


In [ ]:
df_logs_grouped = fdl.utils.pandas.convert_flat_csv_data_to_grouped(input_data=df_logs, group_by_col='srch_id')

In [ ]:
df_logs_grouped.head(2)

### 5.b Publish events

In [ ]:
client.publish_events_batch(project_id=PROJECT_ID,
                            model_id=MODEL_ID,
                            batch_source=df_logs_grouped,
                            id_field='event_id',
                            timestamp_field='time_epoch')

# 7. Get insights


**You're all done!**
  
You can now head to Fiddler URL and start getting enhanced observability into your model's performance. Run the following code block to get your URL:



In [ ]:
print('/'.join([URL, 'projects', PROJECT_ID, 'models', MODEL_ID, 'monitor']))

*Please allow 3-5 minutes for monitoring data to populate the charts.*

--------
**Questions?**  
  
Check out [our docs](https://docs.fiddler.ai/) for a more detailed explanation of what Fiddler has to offer.

Join our [community Slack](http://fiddler-community.slack.com/) to ask any questions!

If you're still looking for answers, fill out a ticket on [our support page](https://fiddlerlabs.zendesk.com/) and we'll get back to you shortly.